<a href="https://colab.research.google.com/github/Nicola-Ibrahim/Pareto-Optimization/blob/main/notebooks/01_pareto_front.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electric Vehicle Control Optimization

## System Definition
We model an electric vehicle (EV) traveling a fixed distance $D$, where we control its acceleration profile to optimize:
1. **Travel Time ($T$)** - Minimize total trip duration
2. **Energy Consumption ($E$)** - Minimize total energy used

## State-Space Representation
| Variable | Symbol | Units   | Description                          |
|----------|--------|---------|--------------------------------------|
| $ x(t) $ | Position | m      | Distance traveled                    |
| $ v(t) $ | Velocity | m/s    | Instantaneous speed                  |
| $ E(t) $ | Energy   | kWh    | Cumulative energy consumption       |

### Control Parameterization
| Phase | Control Input | Symbol | Bounds (m/s²) | Description              |
|-------|---------------|--------|---------------|--------------------------|
| 1     | Acceleration  | $ u_1 $ | [0.3, 4.0]    | Constant acceleration    |
| 2     | Deceleration  | $ u_2 $ | [0.3, 4.0]    | Constant deceleration    |



## System Dynamics
The EV's motion is governed by:

\begin{cases}
\dot{x}(t) = v(t) & \text{(Position)} \\
\dot{v}(t) = \frac{u(t)}{m} - \frac{F_{drag}(v) + F_{roll}}{m} & \text{(Velocity)} \\
F_{drag} = \frac{1}{2}\rho C_d A v^2(t) & \text{(Drag force)} \\
F_{roll} = C_r m g & \text{(Rolling resistance)}
\end{cases}

### Initial Conditions
\begin{aligned}
x(0) = 0,\ v(0) = 0
\end{aligned}

### Terminal Condition
\begin{aligned}
x(T) = D \quad \text{(Must reach target distance)}
\end{aligned}

## Optimization Problem Formulation

### Objectives
\begin{aligned}
\min_{u(t)} \mathbf{J} = \begin{bmatrix} T \\ E \end{bmatrix}
\end{aligned}

1. **Travel Time ($T$)**:
\begin{aligned}
   T = \inf \left\{ t \, \big| \, x(t) = D \right\}
   \end{aligned}
   - $D$: Total travel distance  
   - $x(t)$: Position at time $t$

2. **Energy Consumption ($E$)**:
\begin{aligned}
   E(u) = \int_0^T P\big(u(t), v(t)\big) \, dt
   \end{aligned}
   \begin{aligned}
   P(u,v) = \frac{u(t) \cdot v(t)}{\eta} + P_{\text{aux}}
   \end{aligned}
   - $\eta$: Motor efficiency (0 < $\eta$ ≤ 1)  
   - $P_{\text{aux}}$: Constant auxiliary power  
   - $u(t)$: Control input (acceleration/deceleration)  
   - $v(t)$: Velocity

### Hard Constraints
1. **Control Inputs**:
   $
   u_1 \in [0.3, 4.0]\, \text{m/s}^2,\quad u_2 \in [0.3, 4.0]\, \text{m/s}^2
   $
   
2. **Speed Limit**:
   $
   v(t) \leq v_{max} = 30\, \text{m/s}\quad \forall t\in[0,T]
   $

3. **Battery Limit**:
   $
   E_{total} \leq 20\%, E_{batt} = 0.4\, \text{kWh}
   $

4. **Distance Completion**:
   $
   x(T) = D = 1000\, \text{m} \pm 0.1\%
   $


## Physical Parameters

| Parameter | Symbol | Value | Units | Description |
|-----------|--------|-------|-------|-------------|
| Vehicle mass | \( m \) | 1000 | kg | Total curb weight |
| Drag coefficient | \( C_d \) | 0.24 | - | Aerodynamic profile |
| Frontal area | \( A \) | 2.4 | m² | Cross-sectional area |
| Rolling resistance | \( C_r \) | 0.008 | - | Tire-road friction |
| Motor efficiency | \( \eta_{mtr} \) | 0.85 | - | Drivetrain efficiency |
| Regen efficiency | \( \eta_{regen} \) | 0.70 | - | Braking recovery |
| Air density | \( \rho \) | 1.225 | kg/m³ | Sea level conditions |
| Auxiliary power | \( P_{aux} \) | 300 | W | Electronics load |

## Implementation Notes
- **Discretization**: The continuous control $u(t)$ is parameterized as:
  - $u_1$: Constant acceleration in Phase 1
  - $u_2$: Constant deceleration in Phase 2
- **Numerical Integration**: Trapezoidal rule used for energy calculation
- **Constraints Handling**: Normalized constraints ensure balanced optimization



In [2]:
import numpy as np
from typing import Any, TypedDict
from pathlib import Path
import pickle
import pandas as pd

# Import optimizer modules
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.core.population import Population
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from pymoo.core.result import Result

# Import plotting modules
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots


### EVControl Problem

In [3]:
class EVControlProblem(Problem):
    """
    Multi-objective optimization problem for electric vehicle motion control.
    Optimizes acceleration/deceleration strategies to balance travel time and energy consumption.

    Objectives:
    1. Minimize total travel time (minutes)
    2. Minimize energy consumption (kWh)

    Constraints:
    1. Vehicle speed must not exceed allowed maximum
    2. Total energy used must not exceed battery capacity
    3. Control inputs must stay within acceleration/deceleration limits
    """

    def __init__(self, target_distance_km=10.0) -> None:
        """Initialize vehicle parameters and problem configuration"""

        # ========================
        # Vehicle Physical Parameters
        # ========================
        self.vehicle_mass_kg = 1000            # Total mass of the vehicle
        self.air_density_kg_per_m3 = 1.225     # Air density at standard conditions
        self.drag_coefficient = 0.24           # Aerodynamic shape efficiency
        self.frontal_area_m2 = 2.4             # Cross-sectional area facing airflow
        self.rolling_resistance_coeff = 0.008  # Tire-road friction coefficient
        self.motor_efficiency = 0.85           # Power conversion efficiency (motor)
        self.regenerative_efficiency = 0.70    # Energy recovery efficiency (braking)
        self.auxiliary_power_watt = 300        # Constant power for non-drive systems

        # ========================
        # Operational Constraints
        # ========================
        self.max_speed_mps = 20                # 72 km/h (20 m/s) speed limit
        self.battery_capacity_kwh = 10.0       # Total battery capacity
        self.initial_state_of_charge = 0.9     # Starting charge level (90%)
        self.minimum_state_of_charge = 0.2     # Safety threshold for battery drain

        # Energy availability calculation
        self.available_energy_kwh = (self.initial_state_of_charge -
                                   self.minimum_state_of_charge) * self.battery_capacity_kwh

        # ========================
        # Mission Parameters
        # ========================
        self.target_distance_km = target_distance_km         # Trip distance requirement
        self.target_distance_m = self.target_distance_km * 1000  # Convert to meters

        # ========================
        # Control Input Limits
        # ========================
        self.min_acceleration = 0.2   # m/s² (comfort-focused minimum)
        self.max_acceleration = 2.0   # m/s² (performance-focused maximum)

        # Initialize optimization problem structure
        super().__init__(
            n_var=2,        # Decision variables: [acceleration_rate, deceleration_rate]
            n_obj=2,        # Objectives: [time_minutes, energy_kwh]
            n_constr=3,      # Constraints: [speed_limit, energy_limit, control_bounds]
            xl=np.array([self.min_acceleration, self.min_acceleration]),
            xu=np.array([self.max_acceleration, self.max_acceleration])
        )

    def _clamp_velocity(self, velocity: float) -> float:
        """Ensure velocity stays within physical and regulatory limits"""
        return np.clip(velocity, 0, self.max_speed_mps)

    def _calculate_resistive_forces(self, velocity: float) -> tuple[float, float]:
        """
        Compute opposing forces affecting vehicle motion

        Args:
            velocity: Current vehicle speed in m/s

        Returns:
            tuple: (aerodynamic_drag_force [N], rolling_resistance_force [N])
        """
        aerodynamic_drag = 0.5 * self.air_density_kg_per_m3 * self.drag_coefficient * \
                         self.frontal_area_m2 * velocity**2
        rolling_resistance = self.rolling_resistance_coeff * self.vehicle_mass_kg * 9.81

        return aerodynamic_drag, rolling_resistance

    def _simulate_vehicle_motion(self, acceleration_rate: float,
                                deceleration_rate: float) -> tuple[float, float, float]:
        """
        Simulate vehicle trip through acceleration, cruising, and deceleration phases

        Args:
            acceleration_rate: Control input for acceleration (m/s²)
            deceleration_rate: Control input for braking (m/s²)

        Returns:
            tuple: (total_time_minutes, peak_velocity_mps, actual_distance_m)
        """
        SIMULATION_TIMESTEP = 0.1  # Seconds between state updates

        time_elapsed = 0.0
        distance_traveled = 0.0
        current_velocity = 0.0
        peak_velocity = 0.0

        # Phase 1: Acceleration to cruising speed
        while (current_velocity < self.max_speed_mps and
               distance_traveled < self.target_distance_m):
            current_velocity = self._clamp_velocity(
                current_velocity + acceleration_rate * SIMULATION_TIMESTEP
            )
            distance_traveled += current_velocity * SIMULATION_TIMESTEP
            time_elapsed += SIMULATION_TIMESTEP
            peak_velocity = max(peak_velocity, current_velocity)

        # Phase 2: Constant speed cruising (if distance remains)
        remaining_distance = self.target_distance_m - distance_traveled
        if remaining_distance > 0:
            cruise_duration = remaining_distance / self.max_speed_mps
            time_elapsed += cruise_duration
            distance_traveled = self.target_distance_m  # Fully covered

        # Phase 3: Controlled deceleration to stop
        while distance_traveled < self.target_distance_m and current_velocity > 0:
            current_velocity = self._clamp_velocity(
                current_velocity - deceleration_rate * SIMULATION_TIMESTEP
            )
            distance_traveled += current_velocity * SIMULATION_TIMESTEP
            time_elapsed += SIMULATION_TIMESTEP

        return time_elapsed / 60, peak_velocity, distance_traveled

    def _calculate_energy_consumption(self, acceleration_rate: float,
                                    deceleration_rate: float) -> float:
        """
        Calculate total energy expenditure during the trip

        Args:
            acceleration_rate: Control input for acceleration (m/s²)
            deceleration_rate: Control input for braking (m/s²)

        Returns:
            float: Total energy consumed in kWh
        """
        SIMULATION_TIMESTEP = 0.1  # Seconds between energy calculations
        energy_watt_hours = 0.0
        current_velocity = 0.0
        distance_traveled = 0.0

        while distance_traveled < self.target_distance_m:
            # Determine current operational phase
            if current_velocity < self.max_speed_mps:
                operation_phase = "accelerating"
                current_acceleration = acceleration_rate
            else:
                braking_distance = current_velocity**2 / (2 * deceleration_rate)
                remaining_distance = self.target_distance_m - distance_traveled
                operation_phase = "braking" if remaining_distance <= braking_distance else "cruising"
                current_acceleration = -deceleration_rate if operation_phase == "braking" else 0

            # Calculate power requirements for current state
            drag_force, rolling_force = self._calculate_resistive_forces(current_velocity)

            if operation_phase == "accelerating":
                mechanical_power = (self.vehicle_mass_kg * current_acceleration +
                                  drag_force + rolling_force) * current_velocity
                effective_power = mechanical_power / self.motor_efficiency
            elif operation_phase == "braking":
                mechanical_power = (self.vehicle_mass_kg * current_acceleration +
                                  drag_force + rolling_force) * current_velocity
                effective_power = mechanical_power * self.regenerative_efficiency
            else:  # Cruising
                mechanical_power = (drag_force + rolling_force) * current_velocity
                effective_power = mechanical_power / self.motor_efficiency

            # Sum total energy consumption with auxiliary systems
            total_power = effective_power + self.auxiliary_power_watt
            energy_watt_hours += total_power * SIMULATION_TIMESTEP / 3600  # Convert to Wh

            # Update kinematic state
            current_velocity = self._clamp_velocity(
                current_velocity + current_acceleration * SIMULATION_TIMESTEP
            )
            distance_traveled += current_velocity * SIMULATION_TIMESTEP

        return energy_watt_hours / 1000  # Convert to kWh

    def _evaluate(self, population: np.ndarray, out: dict[str, Any],
                *args, **kwargs) -> None:
        """
        Evaluate a population of candidate solutions

        Args:
            population: Array of candidate solutions (acceleration/deceleration pairs)
            out: Dictionary for storing objectives and constraints
        """
        objective_values = []
        constraint_violations = []

        for accel_rate, decel_rate in population:
            # Simulate vehicle motion with current control parameters
            trip_time, max_speed, final_distance = self._simulate_vehicle_motion(
                accel_rate, decel_rate
            )

            # Calculate energy expenditure
            energy_used = self._calculate_energy_consumption(accel_rate, decel_rate)

            # Calculate constraint violations (normalized)
            speed_violation = max(0, (max_speed - self.max_speed_mps) / self.max_speed_mps)
            energy_violation = max(0, (energy_used - self.available_energy_kwh) /
                                 self.available_energy_kwh)

            # Control input constraint checks
            accel_violation = max(
                (self.min_acceleration - accel_rate)/self.min_acceleration
                if accel_rate < self.min_acceleration else 0,
                (accel_rate - self.max_acceleration)/self.max_acceleration
                if accel_rate > self.max_acceleration else 0
            )
            decel_violation = max(
                (self.min_acceleration - decel_rate)/self.min_acceleration
                if decel_rate < self.min_acceleration else 0,
                (decel_rate - self.max_acceleration)/self.max_acceleration
                if decel_rate > self.max_acceleration else 0
            )
            control_violation = max(accel_violation, decel_violation)

            # Store evaluation results
            objective_values.append([trip_time, energy_used])
            constraint_violations.append([
                speed_violation,
                energy_violation,
                control_violation
            ])


        # Format outputs for optimization framework
        out["F"] = np.array(objective_values)
        out["G"] = np.array(constraint_violations)


## NSGA-II Optimization Progress Metrics

### Table Column Definitions
| Column        | Description                                                                 | Interpretation                                                                 |
|---------------|-----------------------------------------------------------------------------|--------------------------------------------------------------------------------|
| **`n_gen`**   | Current generation number                                                   | Tracks optimization progress (1 → N)                                           |
| **`n_eval`**  | Cumulative function evaluations                                             | Total objective function calls (time + energy calculations)                    |
| **`n_nds`**   | Non-dominated solutions in population                                       | Count of Pareto-optimal solutions found                                        |
| **`cv_min`**  | Minimum constraint violation                                                | `0.0` = feasible solution exists                                               |
| **`cv_avg`**  | Average constraint violation across population                              | Lower values → better feasibility                                              |
| **`eps`**     | Generational improvement (ε-progress)                                       | Measures movement of Pareto front                                              |
| **`indicator`**| Reference point for ε calculation                                          | Shows what drove improvement                                                   |

### Key Metric Details
#### Constraint Violations (`cv_min`, `cv_avg`)
- **`0.000000E+00`**: Perfect feasibility (all constraints satisfied)
- **Decreasing values**: Population is becoming more feasible
- **Example**: `cv_avg=0.565` → Average solution violates constraints by ~56.5%

#### Pareto Front Progress (`eps`)
| Value Range    | Interpretation                          | Example           |
|----------------|----------------------------------------|-------------------|
| **`0.01-0.1`** | Good improvement                       | `0.056` (Gen 4)  |
| **`<0.01`**    | Convergence                            | `0.001`          |
| **`-`**        | No comparison (first generation)       | Gen 1            |

#### Improvement Indicators
| Indicator  | Meaning                                                                 | When It Appears               |
|------------|-------------------------------------------------------------------------|-------------------------------|
| `ideal`    | Progress toward best possible values                                   | Early optimization (Gens 2-4) |
| `nadir`    | Progress away from worst-case values                                   | Late-stage refinement         |
| `f`        | General front movement (mixed criteria)                                | Middle phases                 |

### Your Optimization Status
1. **Feasibility Achieved**: `cv_min=0.0` since Gen 1 → Valid solutions exist  
2. **Rapid Improvement**: Large `eps` drops (0.166 → 0.056) → Effective exploration  
3. **Pareto Growth**: `n_nds` increasing (4 → 63) → Discovering more optimal solutions  
4. **Ideal-Driven**: Early `indicator=ideal` → Focusing on best potential outcomes

In [4]:
class ParetoData(TypedDict):
    """Structure of saved Pareto front data in PKL files"""
    accel: np.ndarray
    decel: np.ndarray
    time: np.ndarray
    energy: np.ndarray
    feasible: np.ndarray
    distance_km: float
    algorithm: str
    config: dict[str, Any]

class OptimizationRunner:
    """Handles NSGA-II optimization process and result archiving"""

    def __init__(self,
                 distance_km: float,
                 vehicle_config: dict[str, Any] = None,
                 population: int = 200,
                 generations: int = 16):
        """
        Args:
            distance_km: Target route distance in kilometers
            vehicle_config: Dictionary of vehicle parameters
            population: NSGA-II population size
            generations: Number of optimization generations
        """
        self.distance_km = distance_km
        self.vehicle_config = vehicle_config or {
            'battery': 10.0,  # kWh
            'max_speed': 20.0,  # m/s
            'mass': 1000  # kg
        }
        self.population = population
        self.generations = generations
        self.result: Result = None

    def run(self) -> 'ParetoResultArchiver':
        """Execute optimization and return archiver with results"""
        problem = EVControlProblem(target_distance_km=self.distance_km)

        algorithm = NSGA2(
            pop_size=self.population,
            crossover=SBX(prob=0.9, eta=15),
            mutation=PolynomialMutation(prob=0.2, eta=20),
            eliminate_duplicates=True
        )

        self.result = minimize(
            problem=problem,
            algorithm=algorithm,
            termination=('n_gen', self.generations),
            seed=42,
            verbose=False
        )

        return ParetoResultArchiver(
            result=self.result,
            distance_km=self.distance_km,
            vehicle_config=self.vehicle_config
        )

class ParetoResultArchiver:
    """Handles processing and saving of complete optimization results"""

    def __init__(self,
                 result: Result,
                 distance_km: float,
                 vehicle_config: dict[str, Any]):
        """
        Args:
            result: Complete optimization result object from pymoo
            distance_km: Target distance for these results
            vehicle_config: Vehicle configuration parameters
        """
        self.result = result
        self.distance_km = distance_km
        self.output_dir: Path = Path("pareto_results")
        self.vehicle_config = vehicle_config

    def _process_data(self) -> dict:
        """Convert raw results to comprehensive structured format"""
        # Get Pareto front indices
        pareto_indices = NonDominatedSorting().do(self.result.F)[0]

        # Full solution data
        all_solutions = {
            'accel_ms2': self.result.X[:, 0],
            'decel_ms2': self.result.X[:, 1],
            'time_min': self.result.F[:, 0],
            'energy_kwh': self.result.F[:, 1],
            'constraint_violations': {
                'speed_violation': self.result.G[:, 0],          # Speed constraint violations
                'energy_violation': self.result.G[:, 1],         # Energy constraint violations
                'control_violation': self.result.G[:, 2]         # Control input violations
            },
            'feasible': self.result.CV <= 0.0,
        }

        # Pareto front solutions
        pareto_front = {
            'accel_ms2': self.result.X[pareto_indices, 0],
            'decel_ms2': self.result.X[pareto_indices, 1],
            'time_min': self.result.F[pareto_indices, 0],
            'energy_kwh': self.result.F[pareto_indices, 1],
            'constraint_violations': {
                'speed_violation': self.result.G[pareto_indices, 0],
                'energy_violation': self.result.G[pareto_indices, 1],
                'control_violation': self.result.G[pareto_indices, 2]
            },
            'feasible': self.result.CV[pareto_indices] <= 0.0,
        }

        # Optimization metadata
        metadata = {
            'distance_km': self.distance_km,
            'algorithm': 'NSGA-II',
            'config': {
                'vehicle': {
                    'battery_kwh': self.vehicle_config['battery'],
                    'max_speed_mps': self.vehicle_config['max_speed'],
                    'mass_kg': self.vehicle_config['mass']
                },
                'optimization': {
                    'population_size': len(self.result.pop),
                    'generations': self.result.algorithm.n_gen,
                    'crossover_prob': 0.9,
                    'mutation_prob': 0.2
                }
            },
        }

        return dict(all_solutions=all_solutions, pareto_front=pareto_front, metadata=metadata)

    def save(self) -> Path:
        """Save complete optimization results to pickle file"""
        self.output_dir.mkdir(exist_ok=True, parents=True)

        processed_data = self._process_data()
        file_path = self.output_dir / f"pareto_{int(self.distance_km)}km.pkl"

        with file_path.open('wb') as f:
            pickle.dump(processed_data, f, protocol=pickle.HIGHEST_PROTOCOL)

        return file_path

In [7]:
# Optimization targets
target_distances_km = [5]

# Run optimizations
for distance in target_distances_km:
    print(f"⚡ Optimizing {distance}km scenario...")
    # Single-line execution
    result = (
        OptimizationRunner(distance_km=distance)
        .run()  # Returns ResultArchiver
        .save() # Returns Path to saved file
    )

⚡ Optimizing 5km scenario...


In [8]:
def load_pareto_front(distance_km, archive_dir):
    file_path = Path(archive_dir) / f"pareto_{int(distance_km)}km.pkl"
    with open(file_path, 'rb') as f:
        return pickle.load(f)

# Example usage
front = load_pareto_front(5, "pareto_results")

front.get('pareto_front').keys()

dict_keys(['accel_ms2', 'decel_ms2', 'time_min', 'energy_kwh', 'constraint_violations', 'feasible'])

In [38]:
# Convert to DataFrame
columns = ['accel_ms2', 'decel_ms2', 'time_min', 'energy_kwh']
df_front = pd.DataFrame(front.get('pareto_front'), columns=columns)
df_front['distance'] = front.get('metadata').get('distance_km')


# Sort by travel time (ascending
df_front

,accel_ms2,decel_ms2,time_min,energy_kwh,distance
0,1.999964,1.976650,4.249168,0.454485,5
1,0.201163,0.200334,4.994351,0.424796,5
2,1.184654,1.961086,4.306522,0.452888,5
3,1.970650,1.968293,4.250410,0.453468,5
4,0.207261,0.200767,4.969974,0.426293,5
...,...,...,...,...,...
195,0.245268,1.979362,4.845363,0.431730,5
196,0.232102,1.982749,4.883908,0.430398,5
197,0.532430,1.952717,4.478864,0.442599,5
198,0.223758,0.208781,4.910686,0.429422,5


In [43]:
df_results.describe()

,accel_ms2,decel_ms2,time_min,energy_kwh,distance
count,200.000000,200.000000,200.000000,200.000000,200.0
mean,0.557949,1.793379,4.611674,0.439003,5.0
std,0.459963,0.474858,0.218830,0.007787,0.0
min,0.201163,0.200334,4.249168,0.424796,5.0
25%,0.255783,1.875259,4.412072,0.432503,5.0
50%,0.367688,1.974996,4.619124,0.437705,5.0
75%,0.676853,1.990069,4.817430,0.444620,5.0
max,1.999964,1.999580,4.994351,0.454485,5.0


## 3. Pareto Front Interpretation
The optimal solutions represent trade-offs:
- **Left Extreme**: Minimum time (aggressive acceleration)
- **Right Extreme**: Minimum energy (gentle acceleration)
- **Middle**: Balanced compromises

The Pareto front $ \mathcal{X}_{\text{Pareto}} $ consists of non-dominated solutions:
\begin{aligned}
\mathcal{X}_{\text{Pareto}} = \left\{ \mathbf{J}_i = [T_i, E_i]^T \, \big| \, \nexists \, \mathbf{J}_j \text{ where } T_j \leq T_i \text{ and } E_j \leq E_i \right\}
\end{aligned}

In [ ]:
# # Extract non-dominated solutions
# F = result.F  # Objective space values (T, E)
# X = result.X  # Decision variables

# # Filter non-dominated solutions
# nds_indices = NonDominatedSorting().do(F)
# pareto_front = F[nds_indices].reshape(200, 2)

# print(f"Found {pareto_front.shape[0]} Pareto-optimal solutions.")


### Visualize problem bounds

In [44]:
# Sort the Pareto front by travel time (x-axis)
sorted_front = df_front.sort_values('time_min')

# Create subplots with 1 row and 2 columns
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Design Space", "Pareto Front"),
                    horizontal_spacing=0.12,
                    column_widths=[0.3, 0.3])

# 1. Design Space Plot (Left)
fig.add_trace(
    go.Scatter(
        x=df_front['accel_ms2'],
        y=df_front['decel_ms2'],
        mode='markers',
        marker=dict(
            size=8,
            color='rgba(255, 100, 100, 0.7)',
            line=dict(width=1, color='DarkRed')
        ),
        name='Design Points',
        hovertemplate="<b>Accel Phase 1</b>: %{x:.2f} m/s²<br><b>Accel Phase 2</b>: %{y:.2f} m/s²<extra></extra>"
    ),
    row=1, col=1
)

# 2. Pareto Front Plot (Right)
fig.add_trace(
    go.Scatter(
        x=df_front['time_min'],
        y=df_front['energy_kwh'],
        mode='markers',
        marker=dict(
            size=8,
            color='rgba(50, 200, 50, 0.8)',
            opacity=0.5,
            line=dict(width=1, color='DarkGreen')
        ),
        name='Pareto Front',
        hovertemplate="<b>Time</b>: %{x:.2f} min<br><b>Energy</b>: %{y:.4f} kWh<extra></extra>"
    ),
    row=1, col=2
)

# Add smooth trend line
fig.add_trace(
    go.Scatter(
        x=sorted_front['time_min'],
        y=sorted_front['energy_kwh'],
        mode='lines',
        line=dict(
            color='black',
            width=3,
            shape='spline',
            smoothing=0.5
        ),
        name='Pareto Frontier',
        hoverinfo='skip',
        opacity=0.5
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="EV Control Optimization Results",
    title_x=0.5,
    title_font=dict(size=20),
    showlegend=False,
    width=1400,
    height=550,
    margin=dict(l=60, r=60, b=80, t=100),
    plot_bgcolor='rgba(248,248,255,1)',
    paper_bgcolor='rgba(248,248,255,1)'
)

# Customize axes
for col in [1, 2]:
    fig.update_xaxes(
        showgrid=True,
        gridcolor='rgba(200,200,200,0.5)',
        zerolinecolor='lightgrey',
        row=1, col=col
    )
    fig.update_yaxes(
        showgrid=True,
        gridcolor='rgba(200,200,200,0.5)',
        zerolinecolor='lightgrey',
        row=1, col=col
    )

# Axis titles
fig.update_xaxes(title_text="Acceleration Phase 1 (m/s²)", row=1, col=1)
fig.update_yaxes(title_text="Acceleration Phase 2 (m/s²)", row=1, col=1)
fig.update_xaxes(title_text="Travel Time (min)", row=1, col=2)
fig.update_yaxes(title_text="Energy Consumption (kWh)", row=1, col=2)

# Add annotations
fig.add_annotation(
    x=0.15, y=0.02,
    xref="paper", yref="paper",
    text="● Design Parameters",
    showarrow=False,
    font=dict(color='darkred')
)
fig.add_annotation(
    x=0.78, y=0.02,
    xref="paper", yref="paper",
    text="◆ Pareto Front",
    showarrow=False,
    font=dict(color='darkgreen')
)

fig.show()

### Visualize the Pareto Front


In [46]:
# Prepare data using DataFrame sources
metadata = front.get('metadata')
battery_kwh = metadata['config']['vehicle']['battery_kwh']

# Create combined array from all solutions (assuming df_all exists)
all_F = np.column_stack((df_front['time_min'], df_front['energy_kwh']))

# Identify non-dominated solutions
nds = NonDominatedSorting()
fronts = nds.do(all_F, only_non_dominated_front=False)
pareto_mask = np.zeros(len(all_F), dtype=bool)
pareto_mask[fronts[0]] = True

# Create figure
fig = go.Figure()

# Add traces with SOC-aware hover text
fig.add_trace(
    go.Scatter(
        x=all_F[~pareto_mask, 0],
        y=all_F[~pareto_mask, 1],
        mode='markers',
        name='Dominated Solutions',
        marker=dict(
            color='lightgray',
            size=6,
            opacity=0.4,
            line=dict(width=0)
        ),
        hovertemplate=(
            "<b>Time</b>: %{x:.2f} min<br>"
            "<b>Energy</b>: %{y*1000:.0f} Wh<br>"
            "<b>SOC Used</b>: %{customdata:.1%}<br>"
            "<i>Dominated solution</i>"
            "<extra></extra>"
        ),
        customdata=all_F[~pareto_mask, 1]/battery_kwh
    )
)

fig.add_trace(
    go.Scatter(
        x=all_F[pareto_mask, 0],
        y=all_F[pareto_mask, 1],
        mode='markers',
        name='Historical Pareto Front',
        marker=dict(
            color='blue',
            size=8,
            opacity=0.7,
            line=dict(width=0)
        ),
        hovertemplate=(
            "<b>Time</b>: %{x:.2f} min<br>"
            "<b>Energy</b>: %{y*1000:.0f} Wh<br>"
            "<b>SOC Used</b>: %{customdata:.1%}<br>"
            "<i>Historical optimal</i>"
            "<extra></extra>"
        ),
        customdata=all_F[pareto_mask, 1]/battery_kwh
    )
)

# Add final Pareto front from df_front
if not df_front.empty:
    fig.add_trace(
        go.Scatter(
            x=df_front['time_min'],
            y=df_front['energy_kwh'],
            mode='markers',
            name='Final Pareto Front',
            marker=dict(
                color='red',
                size=4,
                opacity=1,
                line=dict(color='black', width=1),
            ),
            hovertemplate=(
                "<b>Time</b>: %{x:.2f} min<br>"
                "<b>Energy</b>: %{y*1000:.0f} Wh<br>"
                "<b>SOC Used</b>: %{customdata:.1%}<br>"
                "<i>Final optimal solution</i>"
                "<extra></extra>"
            ),
            customdata=df_front['energy_kwh']/battery_kwh
        )
    )

# Update layout
fig.update_layout(
    title='Evolution of Solutions with Dominated and Non-Dominated Points',
    title_x=0.5,
    title_font=dict(size=24),
    xaxis_title='Travel Time [min]',
    yaxis_title='Energy Consumption [kWh]',
    legend_title='Solution Types',
    width=1000,
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode='closest',
    xaxis=dict(
        gridcolor='lightgray',
        zerolinecolor='lightgray',
        showline=True,
        linewidth=2,
        linecolor='black'
    ),
    yaxis=dict(
        gridcolor='lightgray',
        zerolinecolor='lightgray',
        showline=True,
        linewidth=2,
        linecolor='black'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1,
        bgcolor='rgba(255,255,255,0.8)'
    )
)

# Add annotations for extreme points
if not df_front.empty:
    min_time_point = df_front.loc[df_front['time_min'].idxmin()]
    min_energy_point = df_front.loc[df_front['energy_kwh'].idxmin()]

    fig.add_annotation(
        x=min_time_point['time_min'],
        y=min_time_point['energy_kwh'],
        text="Min Time",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-40
    )

    fig.add_annotation(
        x=min_energy_point['time_min'],
        y=min_energy_point['energy_kwh'],
        text="Min Energy",
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=40
    )

fig.show()

### Running metric

In [ ]:
from pymoo.util.running_metric import RunningMetricAnimation

running = RunningMetricAnimation(delta_gen=5,
                        n_plots=3,
                        key_press=False,
                        do_show=True)

for algorithm in result.history:
    running.update(algorithm)
